<a href="https://colab.research.google.com/github/Anjasfedo/Code-as-a-Cryptography/blob/main/lzw_img.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np
import pickle

# Fungsi untuk mengompresi menggunakan algoritma LZW
def lzw_compress(uncompressed):
    dict_size = 256
    dictionary = {chr(i): i for i in range(dict_size)}

    w = ""
    compressed_data = []

    for c in uncompressed:
        wc = w + c
        if wc in dictionary:
            w = wc
        else:
            compressed_data.append(dictionary[w])
            dictionary[wc] = dict_size
            dict_size += 1
            w = c

    if w:
        compressed_data.append(dictionary[w])

    return compressed_data

# Fungsi untuk dekompresi menggunakan algoritma LZW
def lzw_decompress(compressed):
    dict_size = 256
    dictionary = {i: chr(i) for i in range(dict_size)}

    result = []

    w = chr(compressed.pop(0))
    result.append(w)

    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError("Bad compressed k: %s" % k)

        result.append(entry)
        dictionary[dict_size] = w + entry[0]
        dict_size += 1
        w = entry

    return ''.join(result)

# Mengunduh gambar Lena dari URL
lena_url = "https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png"
response = requests.get(lena_url)
img = Image.open(BytesIO(response.content))

# Simpan gambar yang diunduh sebagai file lokal (misal: 'lena_image.png')
img.save("lena_image.png")
img = img.convert("L")  # Konversi ke grayscale

# Konversi gambar menjadi urutan byte
img_data = np.array(img).flatten()
img_string = ''.join([chr(pixel) for pixel in img_data])

# Kompresi menggunakan LZW
compressed_data = lzw_compress(img_string)

# Simpan data terkompresi ke file binari
with open('compressed_lena_image.bin', 'wb') as f:
    pickle.dump(compressed_data, f)

print("Data terkompresi telah disimpan sebagai 'compressed_lena_image.bin'")

# Dekompresi
with open('compressed_lena_image.bin', 'rb') as f:
    loaded_compressed_data = pickle.load(f)

decompressed_data = lzw_decompress(loaded_compressed_data)

# Konversi kembali ke array numpy
decompressed_img_data = np.array([ord(char) for char in decompressed_data], dtype=np.uint8)
decompressed_img = decompressed_img_data.reshape(img.size[1], img.size[0])

# Simpan hasil dekompresi sebagai gambar
decompressed_img_pil = Image.fromarray(decompressed_img)
decompressed_img_pil.save('decompressed_lena_image.jpg')

print("Gambar hasil dekompresi telah disimpan sebagai 'decompressed_lena_image.jpg'")

Data terkompresi telah disimpan sebagai 'compressed_lena_image.bin'
Gambar hasil dekompresi telah disimpan sebagai 'decompressed_lena_image.jpg'
